<a href="https://colab.research.google.com/github/r45hm33t/Email-Spam-Detection/blob/main/Email_spam_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score
from sklearn.preprocessing import LabelEncoder

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df = pd.read_csv('mails.csv')

In [ ]:
df.sample(5)

,category,message
2147,ham,So can collect ur laptop?
2953,ham,Tell me whos this pls:-)
689,ham,Thanks love. But am i doing torch or bold.
5257,ham,"As usual..iam fine, happy &amp; doing well..:)"
1658,ham,S:-)if we have one good partnership going we w...


In [ ]:
df.shape

(5572, 2)

PREPROCESSING


In [ ]:
encoder = LabelEncoder()

In [ ]:
df['category'] = encoder.fit_transform(df['category'])

In [ ]:
def transform_text(message):
    message = message.lower()
    message = nltk.word_tokenize(message)
    
    y = []
    for i in message:
        if i.isalnum():
            y.append(i)
    
    text = y[:]
    y.clear()
    
    for i in message:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)
            
    message = y[:]
    y.clear()
    ps=PorterStemmer()
    for i in message:
        y.append(ps.stem(i))
    
            
    return " ".join(y)

In [ ]:
transform_text("OH YEAH,AND HAV A GREAT TIME IN NEWQUAY-SEND ME A POSTCARD !1 LOOK AFTER ALL THE GIRLS WHILE IM GONE(U KNOW THE 1IM TALKIN BOUT!)xx")

'oh yeah hav great time newquay-send postcard 1 look girl im gone u know 1im talkin bout xx'

In [ ]:
df['transformed_text'] = df['message'].apply(transform_text)

In [ ]:
df.head()

,category,message,transformed_text
0,0,"Go until jurong point, crazy.. Available only ...",go jurong point crazi .. avail bugi n great wo...
1,0,Ok lar... Joking wif u oni...,ok lar ... joke wif u oni ...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,free entri 2 wkli comp win fa cup final tkt 21...
3,0,U dun say so early hor... U c already then say...,u dun say earli hor ... u c alreadi say ...
4,0,"Nah I don't think he goes to usf, he lives aro...",nah n't think goe usf live around though


MODEL BUILDING

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
cv = CountVectorizer()
tfidf = TfidfVectorizer(max_features=3000)

In [ ]:
tfidf

TfidfVectorizer(max_features=3000)

In [ ]:
X = tfidf.fit_transform(df['transformed_text']).toarray()

In [ ]:
X.shape

(5572, 3000)

In [ ]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
y = df['category'].values

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [ ]:
gnb = GaussianNB()
mnb = MultinomialNB()
bnb = BernoulliNB()

In [ ]:
gnb.fit(X_train,y_train)
y_pred1 = gnb.predict(X_test)
print(accuracy_score(y_test,y_pred1))

0.852914798206278


In [ ]:
mnb.fit(X_train,y_train)
y_pred2 = mnb.predict(X_test)
print(accuracy_score(y_test,y_pred2))

0.9704035874439462


In [ ]:
bnb.fit(X_train,y_train)
y_pred3 = bnb.predict(X_test)
print(accuracy_score(y_test,y_pred3))

0.9775784753363229


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

In [ ]:
svc = SVC(kernel='sigmoid', gamma=1.0)
knc = KNeighborsClassifier()
dtc = DecisionTreeClassifier(max_depth=5)
lrc = LogisticRegression(solver='liblinear', penalty='l1')
rfc = RandomForestClassifier(n_estimators=50, random_state=2)
abc = AdaBoostClassifier(n_estimators=50, random_state=2)
bc = BaggingClassifier(n_estimators=50, random_state=2)
etc = ExtraTreesClassifier(n_estimators=50, random_state=2)
gbdt = GradientBoostingClassifier(n_estimators=50,random_state=2)
xgb = XGBClassifier(n_estimators=50,random_state=2)

In [ ]:
clfs = {
    'SVC' : svc,
    'KN' : knc, 
    'NB': bnb, 
    'DT': dtc, 
    'LR': lrc, 
    'RF': rfc, 
    'AdaBoost': abc, 
    'BgC': bc, 
    'ETC': etc,
    'GBDT':gbdt,
    'xgb':xgb
}

In [ ]:
def train_classifier(clf,X_train,y_train,X_test,y_test):
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test,y_pred)
    
    return accuracy

In [ ]:
train_classifier(svc,X_train,y_train,X_test,y_test)

0.9739910313901345

In [ ]:
accuracy_scores = []

for name,clf in clfs.items():
    
    current_accuracy = train_classifier(clf, X_train,y_train,X_test,y_test)
    
    print("For ",name)
    print("Accuracy - ",current_accuracy)
    
    accuracy_scores.append(current_accuracy)

For  SVC
Accuracy -  0.9739910313901345
For  KN
Accuracy -  0.9022421524663677
For  NB
Accuracy -  0.9775784753363229
For  DT
Accuracy -  0.9345291479820628
For  LR
Accuracy -  0.95695067264574
For  RF
Accuracy -  0.97847533632287
For  AdaBoost
Accuracy -  0.9748878923766816


In [ ]:
performance_df = pd.DataFrame({'Algorithm':clfs.keys(),'Accuracy':accuracy_scores}).sort_values('Accuracy',ascending=False)

In [ ]:
performance_df

In [ ]:
estimators=[('rf', rfc), ('nb', bnb), ('et', etc)]
final_estimator=SVC()
from sklearn.ensemble import StackingClassifier
clf = StackingClassifier(estimators=estimators, final_estimator=final_estimator)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy -",accuracy_score(y_test,y_pred))

In [ ]:
!pip install -q streamlit

In [ ]:
!./ngrok authtokens 2NaORM0YmXMeC1zQrCxAxzA3cQo_7EwhxSCRz3M2AiwxEr67T

In [ ]:
!pip install pyngrok

In [ ]:
from pyngrok import ngrok 
!streamlit run streamlit_app.py&>/dev/null&
public_url = ngrok.connect(port='8501')

In [ ]:
import pickle
pickle.dump(tfidf,open('vectorizes.pkl','wb'))
pickle.dump(clf,open('models.pkl','wb'))
tfidff = pickle.load(open('vectorizes.pkl','rb'))
mode_l = pickle.load(open('models.pkl','rb'))

In [ ]:
mode_l

In [ ]:
%%writefile streamlit_app.py 
import streamlit as st 
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
import pickle
st.markdown(""" This is a Streamlit App """)
tfidff = pickle.load(open('vectorizes.pkl','rb'))
mode_l = pickle.load(open('models.pkl','rb'))
st.title("Email Spam Classifier")
mail_text = st.text_input("Input the email")
def transform_text(message):
    message = message.lower()
    message = nltk.word_tokenize(message)
    
    y = []
    for i in message:
        if i.isalnum():
            y.append(i)
    
    text = y[:]
    y.clear()
    
    for i in message:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)
            
    message = y[:]
    y.clear()
    ps=PorterStemmer()
    for i in message:
        y.append(ps.stem(i))
    
            
    return " ".join(y)
if st.button('predict'):
   transform_mail = transform_text(mail_text) 

   vector_input = tfidff.transform([transform_mail])

   result = mode_l.predict(vector_input)
   if result == 1:
       st.header("spam")
   else:
       st.header("not spam ")

In [ ]:
!streamlit run /content/streamlit_app.py & npx localtunnel  --port 8501